In [3]:

# === 1. Mount Google Drive ===
from google.colab import drive
drive.mount('/content/drive')

# === 2. Load model & tokenizer ===
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Path ke direktori hasil fine-tuning
model_path = "/content/drive/My Drive/Colab Notebooks/Sentry-ID/Level_1/IndoBERTweet/IndoBERTweet_with_aug"

# Load model dan tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()  # set model ke mode evaluasi

# === 3. Siapkan komentar baru untuk demo ===
new_comments = [
    # "Join sekarang slot gacor! minimal depo 10 ribu langsung bisa maxwin cuan besar",  # PROMO_JUDOL
    # "Main judi online bikin rugi, temen gue sampai berhutang",  # RISKY
    # "Bang besok upload konten gaming lagi dong, seru banget kemarin!",  # NORMAL
    # "Yokk ikuttin akuuu, bisa dpt duit cepet di situs inii, no tipu² asli!",  # PROMO_JUDOL
    # "Eh bro, kemarin gue liat iklan betting di IG, bahaya banget",  # RISKY
    # "Wah rejeki nomplok kalau dapet traktiran makan gratis",  # NORMAL
    # "🔥 depo dkit aja udh bisa WD gede, ayo join skrg guys 🔥",  # PROMO_JUDOL
    # "Konten kayak gini bisa nyeret orang ke perjudian lho, hati-hati deh",  # RISKY
    # "Wih kalau gini sih bisa kaya raya gratis tanpa kerja 🤣",  # NORMAL
    # "Mantap bro! kontenmu keren banget, keep going forward!",  # NORMAL
    "Kapan lagi bisa kaya, yokk bisa join langsung di nickame ini yah. di jamin cepat kaya! no hoax",
    "banyak amat komentar judi nya, hapusin dong youtube",
    "bangg main ke banjar yook, biar di traktir"
]

# === 4. Fungsi untuk prediksi ===
def predict_comments(comments):
    inputs = tokenizer(comments, padding=True, truncation=True, max_length=128, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1)
        preds = torch.argmax(probs, dim=-1)

    # Ambil mapping label dari model (tersimpan di config.json)
    id2label = model.config.id2label

    results = []
    for comment, pred_id, prob in zip(comments, preds, probs):
        results.append({
            "komentar": comment,
            "prediksi_label": id2label[int(pred_id)],
            "probabilitas": {id2label[i]: float(prob[i]) for i in range(len(prob))}
        })
    return results

# === 5. Jalankan prediksi dan tampilkan hasil ===
results = predict_comments(new_comments)
for res in results:
    print(f"Komentar: {res['komentar']}")
    print(f"Prediksi Label: {res['prediksi_label']}")
    print(f"Probabilitas: {res['probabilitas']}")
    print("-" * 50)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Komentar: Kapan lagi bisa kaya, yokk bisa join langsung di nickame ini yah. di jamin cepat kaya! no hoax
Prediksi Label: PROMO_JUDOL
Probabilitas: {'NORMAL': 0.0008966900059022009, 'PROMO_JUDOL': 0.998084545135498, 'RISKY': 0.0010188027517870069}
--------------------------------------------------
Komentar: banyak amat komentar judi nya, hapusin dong youtube
Prediksi Label: RISKY
Probabilitas: {'NORMAL': 0.0018846987513825297, 'PROMO_JUDOL': 0.002835730789229274, 'RISKY': 0.9952795505523682}
--------------------------------------------------
Komentar: bangg main ke banjar yook, biar di traktir
Prediksi Label: NORMAL
Probabilitas: {'NORMAL': 0.9947260022163391, 'PROMO_JUDOL': 0.0032159211114048958, 'RISKY': 0.002058113692328334}
--------------------------------------------------
